In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import math
import ast
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

In [2]:
cards = pd.read_csv('TCG_data_all_cards.csv', encoding='latin-1')

In [3]:
#potrzebujemy tej funkcji ze względu na naturę danych — dane są jakby fałszywym słownikiem, więc zmieniamy string na prawdziwy słownik
def parse_legalities(val):
    try:
        if pd.isna(val): return {}
        return ast.literal_eval(val) if isinstance(val, str) else val   #używamy ast.literal_eval dla bezpieczeństwa, gdyby string był błędny lub pusty nie wyrzuca błędu
    except: return {}

df_pixel = cards.copy()

df_pixel['legalities_dict'] = df_pixel['legalities'].apply(parse_legalities)

In [4]:
CARDS_PER_BLOCK = 20    
GAP = 2                 
status_order = ['Standard Legal', 'Expanded Legal', 'Unlimited Legal']
gen_order = ['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth', 'Seventh', 'Eighth', 'Ninth', 'Other']
gen_colors = {
    'First': '#663011', 'Second': '#e41a1c', 'Third': '#ffb060',
    'Fourth': '#ffff33', 'Fifth': '#4daf4a', 'Sixth': '#377eb8',
    'Seventh': '#7b118b', 'Eighth': '#f781bf', 'Ninth': '#250491', 'Other': '#999999'
}

banned_mask = df_pixel['legalities_dict'].apply(lambda d: isinstance(d, dict) and 'Banned' in d.values())

In [5]:
#wykres
fig, ax = plt.subplots(figsize=(16, 9))

current_x_offset = 0
x_ticks_pos = []
max_y_height = 0

patches_by_gen = {gen: [] for gen in gen_order}
all_patches_flat = []

#dzielenie danych na legalności
for status in status_order:
    if status == 'Standard Legal':
        status_data = df_pixel[df_pixel['legalities_dict'].apply(lambda d: isinstance(d, dict) and d.get('standard') == 'Legal')]
    elif status == 'Expanded Legal':
        status_data = df_pixel[df_pixel['legalities_dict'].apply(lambda d: isinstance(d, dict) and d.get('expanded') == 'Legal')]
    elif status == 'Unlimited Legal':
        status_data = df_pixel[df_pixel['legalities_dict'].apply(lambda d: isinstance(d, dict) and d.get('unlimited') == 'Legal')]
    else: status_data = pd.DataFrame()
    
    if status_data.empty: continue
    
    current_grid_width = 18
    block_counter = 0
    #iterujemy po generacjach
    for gen in gen_order:
        gen_subset = status_data[status_data['generation'] == gen]
        num_blocks = math.ceil(len(gen_subset) / CARDS_PER_BLOCK)
        color = gen_colors.get(gen, '#000000')
        for _ in range(num_blocks):
            #obliczanie pozycji (x, y):
            #x: current_x_offset wyznacza nam odpowiednią kolumnę statusu + (block_counter % width) ustawia klocek w poziomie w danej kolumnie
            #y: (block_counter//width) ustawia klocek w pionie
            rect = patches.Rectangle(
                (current_x_offset + (block_counter % current_grid_width), block_counter // current_grid_width), 
                1, 1, linewidth=0.7, edgecolor='white', facecolor=color, visible=False)
            ax.add_patch(rect)
            patches_by_gen[gen].append(rect)
            all_patches_flat.append(rect)
            block_counter += 1
            
    max_y_height = max(max_y_height, (block_counter // current_grid_width) + 1)
    x_ticks_pos.append(current_x_offset + (current_grid_width / 2))
    current_x_offset += current_grid_width + GAP

#ustawienia osi
ax.set_xlim(-GAP, current_x_offset); ax.set_ylim(0, max_y_height + 2)
ax.set_aspect('equal')
ax.set_xticks(x_ticks_pos)
ax.set_xticklabels(status_order, fontsize=12, fontweight='bold') 
ax.set_yticks([])
ax.grid(False) 
for spine in ax.spines.values(): spine.set_visible(False)

#tytuł i text
plt.title("Legalities by Generations", fontsize=23, pad=40, fontweight='bold')
plt.text(0.5, 1.02, f"1 square = {CARDS_PER_BLOCK} cards", transform=ax.transAxes, ha='center', fontsize=14, color='#555555')

#legenda
legend_patches = [plt.Line2D([0], [0], marker='s', color='w', markerfacecolor=gen_colors[g], markersize=15, label=g) for g in gen_order 
                  if g in df_pixel['generation'].unique()]
ax.legend(handles=legend_patches, title="Generations", bbox_to_anchor=(1.01, 1), loc='upper left', frameon=False)


#ANIMACJA

#stały napis, który jednak jest częścią animacji, bo na końcu znika
static_label = ax.text(
    -0.1, 0.9, "Generation:", transform=ax.transAxes, fontsize=30, 
    fontweight='bold', color='black',ha='left',va='center')

#zmieniający się napis
dynamic_label = ax.text(
    0.25, 0.9,"", transform=ax.transAxes,fontsize=30, fontweight='bold', color='black', 
    ha='left',va='center', bbox=dict(
        facecolor='white',edgecolor='black',boxstyle='round,pad=0.3',
        linewidth=4, alpha=0 ))

plt.close()

#szybkońć animacji - ustawienia
FPS = 20
SECONDS_PER_GEN = 2
FRAMES_PER_GEN = FPS * SECONDS_PER_GEN

#ustawienia napisu na końcu
SECONDS_END_HOLD = 0 
HOLD_FRAMES = FPS * SECONDS_END_HOLD
CLEAN_FRAMES = 5 

#liczba klatek na generacje
TOTAL_GENERATION_FRAMES = len(gen_order) * FRAMES_PER_GEN
TOTAL_FRAMES = int(TOTAL_GENERATION_FRAMES + HOLD_FRAMES + CLEAN_FRAMES)


#ta funkcja ustawia nam stan animacji przed jej rozpoczęciem, punkt startowy
def init():
    for p in all_patches_flat: p.set_visible(False)
    static_label.set_text("") 
    dynamic_label.set_text("")
    bbox = dynamic_label.get_bbox_patch()
    if bbox: bbox.set_visible(False)
    return all_patches_flat + [static_label, dynamic_label]

def update(frame):
    #serce animacji, funkcja wywołuje się dla każdej klatki
    if frame < TOTAL_GENERATION_FRAMES:
        gen_idx = frame // FRAMES_PER_GEN   #obliczanie indeksu generacji
        gen_name = gen_order[int(gen_idx)]
        #pobieranie klocków i koloru
        current_gen_patches = patches_by_gen[gen_name]
        current_color_hex = gen_colors[gen_name]
        
        #odsłanianie klocków danej generacji
        for rect in current_gen_patches: rect.set_visible(True)
        
        #ustawienia tekstu
        static_label.set_text("Generation:")
        static_label.set_color('black')
        dynamic_label.set_text(f"{gen_name}")
        dynamic_label.set_color('black')
        
        #ramka
        bbox = dynamic_label.get_bbox_patch()
        if bbox:
            bbox.set_visible(True)
            bbox.set_facecolor('white')     
            bbox.set_edgecolor(current_color_hex)
            bbox.set_alpha(1)               

    #zarzytrzymanie napisu na samym końcu (ostatnia generacja)
    elif frame < TOTAL_GENERATION_FRAMES + HOLD_FRAMES:
        pass 

    #usunięcie (na samym końcu) wszystkich napisów będących w animacji — statycznego i dynamicznego
    else:
        static_label.set_text("")
        dynamic_label.set_text("")
        bbox = dynamic_label.get_bbox_patch()
        if bbox:
            bbox.set_visible(False)
    return all_patches_flat + [static_label, dynamic_label]

#tego potrzebujemy do zapisu animacji — chcemy aby animacja się nie powtarzała, tylko została w swojej ostatecznej fazie (standardowy PillowWriter odtwarza się w pętli)
class PlayOncePillowWriter(PillowWriter):
    def finish(self):
        self._frames[0].save(
            self.outfile, 
            save_all=True, 
            append_images=self._frames[1:], 
            duration=int(1000 / self.fps)
        )

writer = PlayOncePillowWriter(fps=FPS)

anim = FuncAnimation(fig, update, init_func=init, frames=TOTAL_FRAMES, interval=50, blit=True)

print("Zapisywanie GIF'a. To chwilę potrwa...")
anim.save('TCG_Legality_Generation.gif', writer=writer, dpi=100)
print("Zapisane!")
#HTML(anim.to_jshtml())  #to można odkomentować, żeby zobaczyć animacje w programie, ale szybciej jest po prostu zobaczyć zapisany plik

Zapisywanie GIF'a. To chwilę potrwa...
Zapisane!
